In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

# Random Forest

In [ ]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.types import StringType
from pyspark.sql.functions import UserDefinedFunction
from pyspark.ml.feature import Tokenizer,CountVectorizer,StopWordsRemover
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier,RandomForestClassifier,NaiveBayes
from pyspark.ml.tuning import TrainValidationSplit,ParamGridBuilder
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

spark = SparkSession.builder.appName("Spark Dataframe Intro").getOrCreate()

leafDF = spark.read.option("header","False").option("inferSchema","true").csv("leaf.csv")
leafDF = leafDF.withColumnRenamed("_c0","label")
leafDF = leafDF.withColumnRenamed("_c1","Specimen_Number")
leafDF = leafDF.withColumnRenamed("_c2","Eccentricity")
leafDF = leafDF.withColumnRenamed("_c3","Aspect_Ratio")
leafDF = leafDF.withColumnRenamed("_c4","Elongation")
leafDF = leafDF.withColumnRenamed("_c5","Solidity")
leafDF = leafDF.withColumnRenamed("_c6","Stochastic_Convexity")
leafDF = leafDF.withColumnRenamed("_c7","Isoperimetric_Factor")
leafDF = leafDF.withColumnRenamed("_c8","Maximal_Indentation_Depth")
leafDF = leafDF.withColumnRenamed("_c9","Lobedness")
leafDF = leafDF.withColumnRenamed("_c10","Average_Intensity")
leafDF = leafDF.withColumnRenamed("_c11","Average_Contrast")
leafDF = leafDF.withColumnRenamed("_c12","Smoothness")
leafDF = leafDF.withColumnRenamed("_c13","Third_moment")
leafDF = leafDF.withColumnRenamed("_c14","Uniformity")
leafDF = leafDF.withColumnRenamed("_c15","Entropy")

vec = VectorAssembler(inputCols=leafDF.columns[1:],outputCol="features")
leafDF = vec.transform(leafDF)
leafDF = leafDF.select("features","label")
rfClassifier = RandomForestClassifier()
trainDF, testDF = leafDF.randomSplit([0.7,0.3],seed=123)
#BinaryClassificationEvaluator

eva = MulticlassClassificationEvaluator(metricName="accuracy")

params = ParamGridBuilder()\
    .addGrid(rfClassifier.maxDepth, [2,3,4,6,8,10,20,30]) \
    .addGrid(rfClassifier.numTrees, [1,2,3,5,7,10,15,20,30,50])\
    .addGrid(rfClassifier.impurity, ['gini',"entropy"])\
    .build()
"""
validator = TrainValidationSplit(
                                estimatorParamMaps=params,
                                estimator=rfClassifier,
                                evaluator=eva,
                                trainRatio=0.8
                                )
"""
validator = CrossValidator(
                                estimatorParamMaps=params,
                                estimator=rfClassifier,
                                evaluator=eva,
                                numFolds=5,
                                parallelism=4
                          )
model = validator.fit(trainDF)
sonucDF = model.transform(testDF)
basari = eva.evaluate(sonucDF)
nT = model.bestModel.getNumTrees
mD = model.bestModel._java_obj.getMaxDepth()
imp = model.bestModel._java_obj.getImpurity()


In [ ]:
import pandas as pd

d1 = pd.DataFrame({
      "Params":["Num_Trees","Max Depth","Impurtiy","Accuracy"],
      "Values":[nT,mD,imp,basari]})
d1["Classifier"] = "RandomForest"
d1

,Params,Values,Classifier
0,Num_Trees,50,RandomForest
1,Max Depth,8,RandomForest
2,Impurtiy,gini,RandomForest
3,Accuracy,0.6,RandomForest


# # Random Forest with TrainValidationSplit

In [ ]:
spark = SparkSession.builder.appName("Spark Dataframe Intro").getOrCreate()
leafDF = spark.read.option("header","False").option("inferSchema","true").csv("leaf.csv")
leafDF = leafDF.withColumnRenamed("_c0","label")
leafDF = leafDF.withColumnRenamed("_c1","Specimen_Number")
leafDF = leafDF.withColumnRenamed("_c2","Eccentricity")
leafDF = leafDF.withColumnRenamed("_c3","Aspect_Ratio")
leafDF = leafDF.withColumnRenamed("_c4","Elongation")
leafDF = leafDF.withColumnRenamed("_c5","Solidity")
leafDF = leafDF.withColumnRenamed("_c6","Stochastic_Convexity")
leafDF = leafDF.withColumnRenamed("_c7","Isoperimetric_Factor")
leafDF = leafDF.withColumnRenamed("_c8","Maximal_Indentation_Depth")
leafDF = leafDF.withColumnRenamed("_c9","Lobedness")
leafDF = leafDF.withColumnRenamed("_c10","Average_Intensity")
leafDF = leafDF.withColumnRenamed("_c11","Average_Contrast")
leafDF = leafDF.withColumnRenamed("_c12","Smoothness")
leafDF = leafDF.withColumnRenamed("_c13","Third_moment")
leafDF = leafDF.withColumnRenamed("_c14","Uniformity")
leafDF = leafDF.withColumnRenamed("_c15","Entropy")

vec = VectorAssembler(inputCols=leafDF.columns[1:],outputCol="features")
leafDF = vec.transform(leafDF)
leafDF = leafDF.select("features","label")
rfClassifier = RandomForestClassifier()
trainDF, testDF = leafDF.randomSplit([0.7,0.3],seed=123)
#BinaryClassificationEvaluator

eva = MulticlassClassificationEvaluator(metricName="accuracy")

params = ParamGridBuilder()\
    .addGrid(rfClassifier.maxDepth, [2,4,6,8,10,20,30]) \
    .addGrid(rfClassifier.numTrees, [1,3,5,7,10,15,20,30,50,80])\
    .addGrid(rfClassifier.impurity, ['gini',"entropy"])\
    .build()

validator = TrainValidationSplit(
                                estimatorParamMaps=params,
                                estimator=rfClassifier,
                                evaluator=eva,
                                trainRatio=0.8
                                )
"""
validator = CrossValidator(
                                estimatorParamMaps=params,
                                estimator=rfClassifier,
                                evaluator=eva,
                                numFolds=5,
                                parallelism=4
                          )
"""
model2 = validator.fit(trainDF)
sonucDF2 = model2.transform(testDF)
basari2 = eva.evaluate(sonucDF)
nT2 = model2.bestModel.getNumTrees
mD2 = model2.bestModel._java_obj.getMaxDepth()
imp2 = model2.bestModel._java_obj.getImpurity()
d2 = pd.DataFrame({
      "Params":["Num_Trees","Max Depth","Impurtiy","Accuracy"],
      "Values":[nT2,mD2,imp2,basari2]})
d2["Classifier"] = "RandomForest"
d2

,Params,Values,Classifier
0,Num_Trees,80,RandomForest
1,Max Depth,8,RandomForest
2,Impurtiy,entropy,RandomForest
3,Accuracy,0.6,RandomForest


# Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

spark = SparkSession.builder.appName("Spark Dataframe Intro").getOrCreate()
leafDF = spark.read.option("header","False").option("inferSchema","true").csv("leaf.csv")
leafDF = leafDF.withColumnRenamed("_c0","label")
leafDF = leafDF.withColumnRenamed("_c1","Specimen_Number")
leafDF = leafDF.withColumnRenamed("_c2","Eccentricity")
leafDF = leafDF.withColumnRenamed("_c3","Aspect_Ratio")
leafDF = leafDF.withColumnRenamed("_c4","Elongation")
leafDF = leafDF.withColumnRenamed("_c5","Solidity")
leafDF = leafDF.withColumnRenamed("_c6","Stochastic_Convexity")
leafDF = leafDF.withColumnRenamed("_c7","Isoperimetric_Factor")
leafDF = leafDF.withColumnRenamed("_c8","Maximal_Indentation_Depth")
leafDF = leafDF.withColumnRenamed("_c9","Lobedness")
leafDF = leafDF.withColumnRenamed("_c10","Average_Intensity")
leafDF = leafDF.withColumnRenamed("_c11","Average_Contrast")
leafDF = leafDF.withColumnRenamed("_c12","Smoothness")
leafDF = leafDF.withColumnRenamed("_c13","Third_moment")
leafDF = leafDF.withColumnRenamed("_c14","Uniformity")
leafDF = leafDF.withColumnRenamed("_c15","Entropy")

vec = VectorAssembler(inputCols=leafDF.columns[1:],outputCol="features")
leafDF = vec.transform(leafDF)
leafDF = leafDF.select("features","label")
rfClassifier = LogisticRegression()
trainDF, testDF = leafDF.randomSplit([0.7,0.3],seed=123)
#BinaryClassificationEvaluator

eva = MulticlassClassificationEvaluator(metricName="accuracy")

params = ParamGridBuilder()\
    .addGrid(rfClassifier.elasticNetParam,[0.0, 0.5, 1.0])\
    .addGrid(rfClassifier.maxIter,[50, 100])\
    .addGrid(rfClassifier.regParam,[0.01, 0.5, 2.0]) \
    .build()

validator = CrossValidator(
                                estimatorParamMaps=params,
                                estimator=rfClassifier,
                                evaluator=eva,
                                numFolds=5,
                                parallelism=4
                          )
model2 = validator.fit(trainDF)
sonucDF2 = model2.transform(testDF)
basari2 = eva.evaluate(sonucDF2)
enp = model2.bestModel._java_obj.getElasticNetParam()
maxIt = model2.bestModel._java_obj.getMaxIter()
rp    = model2.bestModel._java_obj.getRegParam()
d3 = pd.DataFrame({
      "Params":["elasticNetParam","maxIter","regParam","Accuracy"],
      "Values":[enp,maxIt,rp,basari2]})
d3["Classifier"] = "LogisticRegression"
d3

,Params,Values,Classifier
0,elasticNetParam,0.00,LogisticRegression
1,maxIter,100.00,LogisticRegression
2,regParam,0.01,LogisticRegression
3,Accuracy,0.55,LogisticRegression


# Naive Bayes

In [ ]:
from pyspark.ml.classification import NaiveBayes

spark = SparkSession.builder.appName("Spark Dataframe Intro").getOrCreate()
leafDF = spark.read.option("header","False").option("inferSchema","true").csv("leaf.csv")
leafDF = leafDF.withColumnRenamed("_c0","label")
leafDF = leafDF.withColumnRenamed("_c1","Specimen_Number")
leafDF = leafDF.withColumnRenamed("_c2","Eccentricity")
leafDF = leafDF.withColumnRenamed("_c3","Aspect_Ratio")
leafDF = leafDF.withColumnRenamed("_c4","Elongation")
leafDF = leafDF.withColumnRenamed("_c5","Solidity")
leafDF = leafDF.withColumnRenamed("_c6","Stochastic_Convexity")
leafDF = leafDF.withColumnRenamed("_c7","Isoperimetric_Factor")
leafDF = leafDF.withColumnRenamed("_c8","Maximal_Indentation_Depth")
leafDF = leafDF.withColumnRenamed("_c9","Lobedness")
leafDF = leafDF.withColumnRenamed("_c10","Average_Intensity")
leafDF = leafDF.withColumnRenamed("_c11","Average_Contrast")
leafDF = leafDF.withColumnRenamed("_c12","Smoothness")
leafDF = leafDF.withColumnRenamed("_c13","Third_moment")
leafDF = leafDF.withColumnRenamed("_c14","Uniformity")
leafDF = leafDF.withColumnRenamed("_c15","Entropy")

vec = VectorAssembler(inputCols=leafDF.columns[1:],outputCol="features")
leafDF = vec.transform(leafDF)
leafDF = leafDF.select("features","label")
rfClassifier = NaiveBayes()
trainDF, testDF = leafDF.randomSplit([0.7,0.3],seed=123)
#BinaryClassificationEvaluator

eva = MulticlassClassificationEvaluator(metricName="accuracy")

params = ParamGridBuilder()\
               .addGrid(rfClassifier.smoothing, [0.0, 0.2, 0.4, 0.6, 1.0])\
               .addGrid(rfClassifier.modelType, ["multinomial"])\
               .build()

validator = CrossValidator(
                                estimatorParamMaps=params,
                                estimator=rfClassifier,
                                evaluator=eva,
                                numFolds=5,
                                parallelism=4
                          )
model4 = validator.fit(trainDF)
sonucDF4 = model4.transform(testDF)
basari4 = eva.evaluate(sonucDF4)
smt = model4.bestModel._java_obj.getSmoothing()
d4 = pd.DataFrame({
      "Params":["smt","Accuracy"],
      "Values":[smt,basari4]})
d4["Classifier"] = "NaiveBayes"
d4

,Params,Values,Classifier
0,smt,0.00,NaiveBayes
1,Accuracy,0.04,NaiveBayes


# DecisionTreeClassifier

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

spark = SparkSession.builder.appName("Spark Dataframe Intro").getOrCreate()
leafDF = spark.read.option("header","False").option("inferSchema","true").csv("leaf.csv")
leafDF = leafDF.withColumnRenamed("_c0","label")
leafDF = leafDF.withColumnRenamed("_c1","Specimen_Number")
leafDF = leafDF.withColumnRenamed("_c2","Eccentricity")
leafDF = leafDF.withColumnRenamed("_c3","Aspect_Ratio")
leafDF = leafDF.withColumnRenamed("_c4","Elongation")
leafDF = leafDF.withColumnRenamed("_c5","Solidity")
leafDF = leafDF.withColumnRenamed("_c6","Stochastic_Convexity")
leafDF = leafDF.withColumnRenamed("_c7","Isoperimetric_Factor")
leafDF = leafDF.withColumnRenamed("_c8","Maximal_Indentation_Depth")
leafDF = leafDF.withColumnRenamed("_c9","Lobedness")
leafDF = leafDF.withColumnRenamed("_c10","Average_Intensity")
leafDF = leafDF.withColumnRenamed("_c11","Average_Contrast")
leafDF = leafDF.withColumnRenamed("_c12","Smoothness")
leafDF = leafDF.withColumnRenamed("_c13","Third_moment")
leafDF = leafDF.withColumnRenamed("_c14","Uniformity")
leafDF = leafDF.withColumnRenamed("_c15","Entropy")

vec = VectorAssembler(inputCols=leafDF.columns[1:],outputCol="features")
leafDF = vec.transform(leafDF)
leafDF = leafDF.select("features","label")
rfClassifier = DecisionTreeClassifier()
trainDF, testDF = leafDF.randomSplit([0.7,0.3],seed=123)
#BinaryClassificationEvaluator

eva = MulticlassClassificationEvaluator(metricName="accuracy")
dtparamGrid = (ParamGridBuilder()
             .addGrid(rfClassifier.maxDepth, [2,3,4, 5, 10, 20, 30])
             .addGrid(rfClassifier.maxBins, [10, 20,30,32,35, 40, 80, 100])
             .build())

validator = CrossValidator(
                                estimatorParamMaps=params,
                                estimator=rfClassifier,
                                evaluator=eva,
                                numFolds=5,
                                parallelism=4
                          )
model5 = validator.fit(trainDF)
sonucDF5 = model5.transform(testDF)
basari5 = eva.evaluate(sonucDF5)
mb = model5.bestModel.getMaxBins()
mD = model5.bestModel._java_obj.getMaxDepth()
d5 = pd.DataFrame({
      "Params":["maxDepth","maxBins","Accuracy"],
      "Values":[mD,mb,basari5]})
d5["Classifier"] = "DecisionTreeClassifier"
d5

,Params,Values,Classifier
0,maxDepth,5.00,DecisionTreeClassifier
1,maxBins,32.00,DecisionTreeClassifier
2,Accuracy,0.35,DecisionTreeClassifier


# Results

In [ ]:
pd.concat([d1,d3,d4,d5], axis=0)

,Params,Values,Classifier
0,Num_Trees,50,RandomForest
1,Max Depth,8,RandomForest
2,Impurtiy,gini,RandomForest
3,Accuracy,0.6,RandomForest
0,elasticNetParam,0,LogisticRegression
1,maxIter,100,LogisticRegression
2,regParam,0.01,LogisticRegression
3,Accuracy,0.55,LogisticRegression
0,smt,0,NaiveBayes
1,Accuracy,0.04,NaiveBayes


the best classification method for the data is random forest